In [1]:
# !pip install matplotlib
# !pip install streamlit
# !pip install pandas
# !pip install seaborn
!pip install sqlalchemy
!pip install pymysql

^C



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%%writefile app.py
import matplotlib
import matplotlib.pyplot as plt
import pymysql
import streamlit as st
import seaborn as sns
import pandas as pd
from mysql.connector import connect, Error
from sqlalchemy import create_engine, text

# 한글 폰트 설정
matplotlib.rc('font', family='Malgun Gothic')


# MySQL 커넥션 (기존 mysql.connector용)
class CarRecommendation:
    def __init__(self, host='localhost', user='root', password='root1234', database='car'):
        self.conn = None
        self.cursor = None
        self.args = {'host': host, 'user': user, 'password': password, 'database': database}

    def connection(self):
        try:
            self.conn = connect(**self.args)
            self.cursor = self.conn.cursor(dictionary=True)
            print('DB 연결 성공')
        except Exception as e:
            print(f'연결 오류: {e}')

    def disconnect(self):
        if self.cursor:
            self.cursor.close()
        if self.conn:
            self.conn.close()
        print('DB 연결 종료')

# ---------------------------
# SQLAlchemy를 이용한 MySQL 연결 설정 (pymysql 사용)
host = "localhost"
user = "root"
password = "root1234"
database = "danawa_car"

# SQLAlchemy 엔진 생성 (pymysql 드라이버 사용)
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
# ---------------------------


# 사용자 맞춤 추천 함수 (SQLAlchemy, pymysql 연결 사용)
def find_best_cars(family_size, budget, hobbies):
    with engine.connect() as conn:
        condition_query = text("""
            SELECT condition_id, car_type_id FROM car_conditions
            WHERE family_size = :family_size
        """)
        conditions = conn.execute(condition_query, {"family_size": family_size}).fetchall()
        if not conditions:
            print("해당 가족 인원수에 맞는 차량 조건이 없습니다.")
            return None
        
        condition_ids = [row[0] for row in conditions]
        car_type_ids = [row[1] for row in conditions]

        hobby_query = text("""
            SELECT DISTINCT condition_id FROM car_condition_hobbies
            WHERE hobby_id IN (SELECT hobby_id FROM hobbies WHERE hobby_name IN :hobbies)
        """)
        hobby_conditions = conn.execute(hobby_query, {"hobbies": tuple(hobbies)}).fetchall()
        hobby_condition_ids = [row[0] for row in hobby_conditions]
        
        matched_conditions = list(set(condition_ids) & set(hobby_condition_ids)) if hobby_condition_ids else condition_ids

        if not matched_conditions:
            print("취미 조건에 맞는 차량이 없지만, 가족 조건에 맞는 차량을 추천합니다.")
            matched_conditions = condition_ids

        car_query = text("""SELECT * FROM cars
            WHERE car_type_id IN :car_type_ids
            AND price <= :budget
            ORDER BY price DESC, efficiency DESC
            LIMIT 5
        """)
        best_cars = conn.execute(car_query, {"car_type_ids": tuple(car_type_ids), "budget": budget}).fetchall()

        if not best_cars:
            print("해당 조건에 맞는 차량이 없습니다.")
            return None
        
        # 만약 차량이 3개 미만이면 추가 추천 (예산보다 20% 높은 차량도 포함)
        if len(best_cars) < 3:
            extra_query = text("""
                SELECT car_name, brand, price, efficiency, horsepower FROM cars
                WHERE car_type_id IN :car_type_ids
                AND price <= :budget * 1.2  -- 예산보다 20% 높은 차량도 포함
                ORDER BY price ASC, efficiency DESC
                LIMIT :needed
            """)
            extra_cars = conn.execute(extra_query, {"car_type_ids": tuple(car_type_ids), "budget": budget, "needed": 3 - len(best_cars)}).fetchall()
            best_cars += extra_cars
        return best_cars

def main():
    st.title('🚗 취미 & 조건별 자동차 추천')

    # 세션 상태로 DB 연결 유지 (mysql.connector용)
    if 'car_rec' not in st.session_state:
        st.session_state['car_rec'] = CarRecommendation()
        st.session_state['car_rec'].connection()
    car_rec = st.session_state['car_rec']

    # 사이드바 메뉴
    main_menu = st.sidebar.radio("메뉴 선택", ['🏠홈', '🔍조회하기', '🚗추천받기'])

    if main_menu == '🏠홈':
        st.subheader('🏠 메인 화면')
        st.write('안녕하세요! 차량 추천 시스템에 오신 것을 환영합니다.')
        st.write('내 라이프스타일과 비슷한 사람들이 어떤 차를 타는지 알아보세요!')

    elif main_menu == '🔍조회하기':
        sub_menu = st.sidebar.selectbox('조회 기준 선택', ['취미별 조회', '가족 구성 별 조회', '직종 별 조회'])

        if sub_menu == '취미별 조회':
            st.subheader('🎭 취미별 차량 선택')
            hobbies_list = ['골프', '낚시', '독서', '등산', '배드민턴', '비디오 감상', '사진촬영', '산책', '수영', '스포츠 관람',
                            '여행', '요리', '음악감상', '자전거 타기', '컴퓨터 게임', '캠핑', '테니스', '헬스', '기타 야외 활동']
            selected = st.multiselect("취미를 3개 선택해주세요.", hobbies_list)
            new_or_used = st.radio('차량 종류 선택', ['새차', '중고차'])

            if len(selected) < 3:
                st.warning("취미를 3개 선택해주세요.")
            elif len(selected) > 3:
                st.error("취미는 3개만 선택할 수 있습니다.")
            else:
                st.success("취미를 정상적으로 선택하셨습니다.")
                conditions = [f"('{hobby}' IN (hobby1, hobby2, hobby3, hobby4, hobby5, hobby6, hobby7, hobby8, hobby9, hobby10))" for hobby in selected]
                where_clause = " AND ".join(conditions) + f' AND used = "{new_or_used}"'

                query = f"""
                SELECT type, fuel, used, COUNT(*) AS freq 
                FROM car_info 
                WHERE {where_clause} 
                GROUP BY fuel, used, type 
                ORDER BY freq DESC 
                LIMIT 3;
                """

                cursor = car_rec.conn.cursor(dictionary=True)
                cursor.execute(query)
                results = cursor.fetchall()
                cursor.close()

                if results:
                    df = pd.DataFrame(results)
                    st.dataframe(df)
                    fig, ax = plt.subplots(figsize=(8, 5))
                    sns.barplot(x='type', y='freq', data=df, palette='coolwarm', ax=ax)
                    ax.set_xlabel('차량 종류')
                    ax.set_ylabel('선택 빈도')
                    ax.set_title('취미별 선택 차량')
                    st.pyplot(fig)
                else:
                    st.info("조건에 맞는 데이터가 없습니다.")

        elif sub_menu == '가족 구성 별 조회':
            st.subheader('👨‍👩‍👧‍👦 가족 구성 별 차량 선택')
            age_group = st.selectbox("연령대 선택", ['20대', '30대', '40대', '50대', '60대 이상'])
            family_type = st.selectbox("가족 구성 선택", ['1인 가구', '신혼기', '자녀 성장기', '자녀 독립기'])
            new_or_used = st.radio('차량 종류 선택', ['새차', '중고차'])

            if st.button("조회하기", key='family'):
                where_clause = f"age = '{age_group}' AND family_type = '{family_type}' AND used = '{new_or_used}'"
                query = f"""
                SELECT type, fuel, used, COUNT(*) AS freq 
                FROM car_info 
                WHERE {where_clause} 
                GROUP BY fuel, used, type 
                ORDER BY freq DESC 
                LIMIT 3;
                """

                cursor = car_rec.conn.cursor(dictionary=True)
                cursor.execute(query)
                results = cursor.fetchall()
                cursor.close()

                if results:
                    df = pd.DataFrame(results)
                    st.dataframe(df)
                    fig, ax = plt.subplots(figsize=(8, 5))
                    sns.barplot(x='type', y='freq', data=df, palette='coolwarm', ax=ax)
                    ax.set_xlabel('차량 종류')
                    ax.set_ylabel('선택 빈도')
                    ax.set_title('가족 구성 별 선택 차량')
                    st.pyplot(fig)
                else:
                    st.info("조건에 맞는 데이터가 없습니다.")

        elif sub_menu == '직종 별 조회':
            st.subheader('💼 직종 별 차량 선택')
            income_level = st.selectbox("월 소득 선택", ['100만원 이하', '500만원 내외', '1000만원 내외', '2000만원 이상'])
            occupation = st.selectbox("직업 선택", ['사무직', '자영업', '학생', '기술직', '전문직'])
            new_or_used = st.radio('차량 종류 선택', ['새차', '중고차'])

            if st.button("조회하기", key='occupation'):
                where_clause = f"income = '{income_level}' AND occupation = '{occupation}' AND used = '{new_or_used}'"
                query = f"""
                SELECT type, fuel, used, COUNT(*) AS freq 
                FROM car_info 
                WHERE {where_clause} 
                GROUP BY fuel, used, type 
                ORDER BY freq DESC 
                LIMIT 3;
                """

                cursor = car_rec.conn.cursor(dictionary=True)
                cursor.execute(query)
                results = cursor.fetchall()
                cursor.close()

                if results:
                    df = pd.DataFrame(results)
                    st.dataframe(df)
                    fig, ax = plt.subplots(figsize=(8, 5))
                    sns.barplot(x='type', y='freq', data=df, palette='coolwarm', ax=ax)
                    ax.set_xlabel('차량 종류')
                    ax.set_ylabel('선택 빈도')
                    ax.set_title('직종 별 선택 차량')
                    st.pyplot(fig)
                else:
                    st.info("조건에 맞는 데이터가 없습니다.")

    elif main_menu == '🚗추천받기':
        st.subheader('사용자 맞춤형 추천')
        menu_2 = st.sidebar.radio("추천 방식 선택", ["사용자 맞춤 추천"])
        
        if menu_2 == "사용자 맞춤 추천":
            family_size = st.number_input("가족 인원수를 입력하세요", min_value=1, value=3, step=1)
            budget = st.number_input("예산을 입력하세요 (단위: 만원)", min_value=0, value=5000  , step=100)
            # 취미 리스트 재사용 (필요시 수정)
            hobbies_list = ['골프', '낚시', '독서', '등산', '배드민턴', '비디오 감상', '사진촬영', '산책', '수영', '스포츠 관람',
                            '여행', '요리', '음악감상', '자전거', '컴퓨터 게임', '캠핑', '테니스', '헬스', '기타 야외 활동']
            selected_hobbies = st.multiselect("취미를 선택하세요", hobbies_list)
            
            if st.button("추천 받기", key='recommend'):
                if not selected_hobbies:
                    st.warning("하나 이상의 취미를 선택해주세요.")
                else:
                    best_cars = find_best_cars(family_size, budget, selected_hobbies)
                    if not best_cars:
                        st.info("❌ 해당 조건에 맞는 차량이 없습니다.")
                    else:
                        df = pd.DataFrame(best_cars)
                        # 추가 시각화나 정보 표시 가능
                        df['image'] = df['image_url'].apply(lambda url: f'<img src="{url}" width="100">' if pd.notnull(url) else '')
                        df.drop(['car_id', 'car_type_id','image_url'], axis=1, inplace=True)
                        df = df[['image', 'brand', 'car_name', 'fuel_type', 'engine_capacity', 'efficiency', 'price', 'release_date']]
                        st.markdown(df.to_html(escape=False, index=False), unsafe_allow_html=True)


    if st.sidebar.button('DB 연결 종료'):
        car_rec.disconnect()
        del st.session_state['car_rec']
        st.write('DB 연결을 종료합니다.')

if __name__ == '__main__':
    main()


Overwriting app.py


In [ ]:
!streamlit run app.py